In [7]:
from gigpy import _utils as ut
import numpy as np
import os


from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor, as_completed


def parallel_process(array, function, n_jobs=16, use_kwargs=False):
    """
        A parallel version of the map function with a progress bar. 

        Args:
            array (array-like): An array to iterate over.
            function (function): A python function to apply to the elements of array
            n_jobs (int, default=16): The number of cores to use
            use_kwargs (boolean, default=False): Whether to consider the elements of array as dictionaries of 
                keyword arguments to function 
        Returns:
            [function(array[0]), function(array[1]), ...]
    """
    
    #If we set n_jobs to 1, just run a list comprehension. This is useful for benchmarking and debugging.
    if n_jobs==1:
        return [function(**a) if use_kwargs else function(a) for a in tqdm(array[:])]
    #Assemble the workers
    with ProcessPoolExecutor(max_workers=n_jobs) as pool:
        #Pass the elements of array into function
        if use_kwargs:
            futures = [pool.submit(function, **a) for a in array[:]]
        else:
            futures = [pool.submit(function, a) for a in array[:]]
        kwargs = {
            'total': len(futures),
            'unit': 'it',
            'unit_scale': True,
            'leave': True
        }
        #Print out the progress as tasks complete
        #for f in tqdm(as_completed(futures), **kwargs):
        #    pass
        
    out = []
    #Get the results from the futures. 
    for i, future in tqdm(enumerate(futures)):
        try:
            out.append(future.result())
        except Exception as e:
            out.append(e)
    return out

def download_atm_unavco_para(data0):
    date0, path0 = data0
    ut.download_atm_unavco(date0,path0) 
    return

def download_atm_unr_para(data0):
    date0,stationName0,path0 = data0
    ut.download_atm_unr(date0,stationName0,path0)  
    return

date_list = ['20180101','20180102','20180103']
path = os.getcwd()
data_all = []
for i in range(len(date_list)):
    data0 = [date_list[i],path]
    data_all.append(data0)

station_unr_list = ut.get_stationlist_atm_unr('20180101')
    
data_all = []
station  = station_unr_list[0:10]
for i in range(len(station)):
    data0 = [date_list[0],station[i],path]
    data_all.append(data0)
#parallel_process(data_all, download_atm_unavco_para, n_jobs=3)




#print(station_unr_list)
print(len(station_unr_list))
#parallel_process(data_all, download_atm_unr_para, n_jobs=4)
with ProcessPoolExecutor(max_workers=4) as pool:
    futures = [pool.submit(download_atm_unr_para, a) for a in data_all]

10565
